In [ ]:
import numpy as np
import os
import librosa.display
import matplotlib.pyplot as plt
from scipy.fftpack import fft
import numpy as np
import librosa
import tensorflow as tf
from keras.utils import np_utils
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input, layers
from tensorflow.keras.layers import Dense, Dropout, LSTM,Flatten
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import glob
from sklearn import preprocessing
from keras.layers import Convolution2D, MaxPooling2D
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle

In [ ]:
data = np.load('/content/drive/MyDrive/warblrb10k_public.npy')
label = np.genfromtxt('/content/drive/MyDrive/warblrb10k_public.txt')

data_2 = np.load('/content/drive/MyDrive/ff1010bird.npy')
label_2 = np.genfromtxt('/content/drive/MyDrive/ff1010bird_metadata_2018.txt', delimiter=',')

real_test_data = np.load('/content/drive/MyDrive/test_data.npy')


In [ ]:
label_1000 = label[:5000]
data_1000 = data[:5000,:,:]
label_2= label_2[:,1]
label_2_1000 = label_2[:5000]

data_2_1000 = data_2[:5000,:,:]

training_data = np.concatenate((data_1000,data_2_1000), axis=0)
training_data = training_data[..., np.newaxis]
print(training_data.shape)
training_labels = np.concatenate((label_1000,label_2_1000), axis=0)
print(training_labels.shape)

(10000, 40, 862, 1)
(10000,)


In [ ]:
csv72 = pd.read_csv('/content/drive/MyDrive/out72345.csv')
csv72 = csv72['Predicted']
csv72 = np.array(csv72)

index_data_ones = np.where(csv72>0.97)
added_ones = real_test_data[index_data_ones]

index_data_zeros = np.where(csv72<0.045)
added_zeros = real_test_data[index_data_zeros]

added_ones = added_ones[..., np.newaxis]
added_zeros = added_zeros[..., np.newaxis]

added_labels_ones = np.ones(len(added_ones))
added_labels_zeros = np.zeros(len(added_zeros))

unsupervised_data = np.concatenate((added_ones,added_zeros), axis=0)

unsupervised_labels = np.concatenate((added_labels_ones,added_labels_zeros), axis=0)
unsupervised_data.shape


(2216, 40, 862, 1)

In [ ]:

data_2000 = data[5001:,:,:]
data_2_2000 = data_2[5001:,:,:]
test_data = np.concatenate((data_2000,data_2_2000), axis=0)
test_data = test_data[..., np.newaxis]

test_data = np.concatenate((test_data,unsupervised_data), axis=0)

label_2000 = label[5001:]
label_2_2000 = label_2[5001:]

test_labels = np.concatenate((label_2000,label_2_2000), axis=0)
test_labels = np.concatenate((test_labels,unsupervised_labels), axis=0)


In [ ]:
test_data.shape

(10120, 40, 862, 1)

In [ ]:
one = np.count_nonzero(test_labels == 1, axis=0) 
print(one)

4840


In [ ]:

test_train_data = np.concatenate((training_data,test_data), axis=0)
test_train_labels = np.concatenate((training_labels,test_labels), axis=0)


In [ ]:

print(test_train_data.shape)
print(test_train_labels.shape)

(20120, 40, 862, 1)
(20120,)


In [ ]:
test_train_data, test_train_labels = shuffle(test_train_data, test_train_labels, random_state=0)

In [ ]:

output_size = 2

model = tf.keras.Sequential([
                            tf.keras.layers.Conv2D(96, kernel_size=(5,5), input_shape=(40,862,1), padding='same', activation='relu'),
                            tf.keras.layers.BatchNormalization(),
                            tf.keras.layers.MaxPool2D(pool_size=(5,5)),
                            tf.keras.layers.Dropout(0.3),  
                            
                            tf.keras.layers.Conv2D(96, kernel_size=(5,5), padding='same',activation='relu'),
                            tf.keras.layers.BatchNormalization(),
                            tf.keras.layers.MaxPool2D(pool_size=(2,2)),
                            tf.keras.layers.Dropout(0.3), 
                           
                            tf.keras.layers.Conv2D(96, kernel_size=(5,5),  padding='same', activation='relu'),
                            tf.keras.layers.BatchNormalization(),
                            tf.keras.layers.MaxPool2D(pool_size=(2,2)),
                            tf.keras.layers.Dropout(0.3),
                             
                            tf.keras.layers.Conv2D(96, kernel_size=(5,5),  padding='same', activation='relu'),
                            tf.keras.layers.BatchNormalization(),
                            tf.keras.layers.MaxPool2D(pool_size=(2,2)),
                            tf.keras.layers.Dropout(0.3),

                            tf.keras.layers.Conv2D(96, kernel_size=(5,5),  padding='same', activation='relu'),
                             
                            tf.keras.layers.Reshape((21, 96), input_shape=(1,21,96)),
                            tf.keras.layers.LSTM(96, return_sequences=True),
                            tf.keras.layers.LSTM(96, return_sequences=True),
                            #tf.keras.layers.LSTM(128, return_sequences=True),
                            tf.keras.layers.Flatten(),
                            tf.keras.layers.Dense(256, activation='relu'),
                            tf.keras.layers.BatchNormalization(),
                            tf.keras.layers.Dense(128, activation='relu'),
                            tf.keras.layers.BatchNormalization(),
                            tf.keras.layers.Dense(64, activation='relu'),
                            tf.keras.layers.Dense(output_size, activation='softmax')    
                            ])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 40, 862, 96)       2496      
_________________________________________________________________
batch_normalization (BatchNo (None, 40, 862, 96)       384       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 8, 172, 96)        0         
_________________________________________________________________
dropout (Dropout)            (None, 8, 172, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 172, 96)        230496    
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 172, 96)        384       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 86, 96)         0

In [ ]:
'''
from tensorflow import keras
opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
'''

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
epochs = 45
model.fit(training_data, training_labels, batch_size=32, epochs=epochs, verbose=1)      


#test_train_data, test_train_labels = shuffle(test_train_data, test_train_labels, random_state=0)
#model.fit(test_train_data, test_train_labels, batch_size=32, epochs=epochs, verbose=1)

Epoch 1/45
313/313 [==============================] - 23s 73ms/step - loss: 0.5388 - accuracy: 0.7447
Epoch 2/45
313/313 [==============================] - 23s 73ms/step - loss: 0.4773 - accuracy: 0.7884
Epoch 3/45
313/313 [==============================] - 23s 72ms/step - loss: 0.4682 - accuracy: 0.7935
Epoch 4/45
313/313 [==============================] - 23s 73ms/step - loss: 0.4670 - accuracy: 0.7916
Epoch 5/45
313/313 [==============================] - 23s 73ms/step - loss: 0.4264 - accuracy: 0.8121
Epoch 6/45
313/313 [==============================] - 23s 73ms/step - loss: 0.3996 - accuracy: 0.8276
Epoch 7/45
313/313 [==============================] - 23s 74ms/step - loss: 0.3772 - accuracy: 0.8341
Epoch 8/45
313/313 [==============================] - 23s 73ms/step - loss: 0.3601 - accuracy: 0.8435
Epoch 9/45
313/313 [==============================] - 23s 73ms/step - loss: 0.3495 - accuracy: 0.8484
Epoch 10/45
313/313 [==============================] - 23s 73ms/step - loss: 0.346

In [ ]:
model.fit(test_data, test_labels, batch_size=32, epochs=epochs, verbose=1)  

In [ ]:
results = model.predict(test_data)

predicted = results[:,1]
#predicted = np.argmax(results,axis=1)
#predicted = np.amax(results,axis=1)
from sklearn.metrics import roc_auc_score
roc_score = roc_auc_score(test_labels,predicted)
print(roc_score)

0.944801406660105


In [ ]:
test_frame = {'Zero' : results[:,0],'One' : results[:,1]}
percentages = pd.DataFrame(data = test_frame)
print(percentages)

In [ ]:
model.evaluate(test_data, test_labels)

In [ ]:
results = model.predict(test_train_data)

predicted = np.amax(results,axis=1)
from sklearn.metrics import roc_auc_score
roc_score = roc_auc_score(test_train_labels,predicted)
print(roc_score)

In [ ]:
test_frame = {'Zero' : results[:,0],'One' : results[:,1]}
percentages = pd.DataFrame(data = test_frame)
print(percentages)

In [ ]:
model.evaluate(test_train_data, test_train_labels)

In [ ]:
real_test_data = real_test_data[..., np.newaxis]

real_test_data.shape

(4512, 40, 862, 1)

In [ ]:
results = model.predict(real_test_data)

In [ ]:
#predicted1 = np.argmax(results,axis=1)
predicted = results[:,1]
print(predicted.shape)

one = np.count_nonzero(predicted > 0.8, axis=0) 
print(one)
two = np.count_nonzero(predicted > 0.5, axis=0) 
print(two)
three = np.count_nonzero(predicted < 0.5, axis=0) 
print(three)
four = np.count_nonzero(predicted < 0.2, axis=0) 
print(four)

(4512,)
1024
1114
3398
3308


In [ ]:
test_name = np.arange(0,len(real_test_data))

frame = {'ID' : test_name, 'Predicted' : predicted }
end_data = pd.DataFrame(data = frame)
# end_data['ID'] = end_data.Id.str.split('.')
# end_data['ID'] = end_data.Id.str.get(0)

compression_opts = dict(method='zip',
                        archive_name='out.csv') 
end_data.to_csv('out.zip', index=False,
          compression=compression_opts) 